# Task3_QR-Code_Based Control

## *1.Design and print your own QR codes*
#### Preparation
1. install qrcode library with pip3
    ```bash
        pip3 install qrcode
   ```

#### implement code

In [ ]:
import qrcode
import cv2

# the most simple way
# qr = qrcode.make("hello world")
# qr.save('my_qr_code_png')

'''
https://pypi.org/project/qrcode/
@param
version (size_qr_code) int[1,40]
error_correction [7,15,30]%
box_size (size_img)
border (size_border)
'''

qr = qrcode.QRCode(
    version=1,
    box_size=15,
    border=5
)

# QR Code data design
data = "POINT_3"

# append data
qr.add_data(data)
# auto fit QR config
qr.make(fit=True)

# set QR color and save
img = qr.make_image(fill='black', back_color='white')

img.save(f'resource/{data}.png')

# put Text on image
bk_img = cv2.imread(f'resource/{data}.png')
cv2.putText(bk_img, str(data), (10, 40), cv2.FONT_HERSHEY_SIMPLEX,
            1, (0, 0, 255), 2, cv2.LINE_AA)
cv2.imshow(str(data), bk_img)
cv2.waitKey()

#cv2.imwrite("resource/Add_Text.png", bk_img)


## *2.Test a QR code detector based on still image*
#### Preparation
1. install pyzbar library with pip3
    ```bash
        pip3 install pyzbar
   ```
    \* if error occur , try this command before pip install
    ```bash
      sudo apt install libzbar0
    ```

#### implement code

In [ ]:
import cv2
import numpy as np
import pyzbar.pyzbar as pyzbar  # sudo apt install libzbar0

path = "resource/HELLO_WORLD.png"

# read still image
image = cv2.imread(path)

# decode with pyzbar API
decodeObjects = pyzbar.decode(image)

# print QR code result
for obj in decodeObjects:
    print("Type:", obj.type)
    print("Data:", obj.data, "\n")


## *3.Test your QR code decoder from the camera stream in real time*
#### embedded decode program into tello stream

In [ ]:
from djitellopy import Tello
from time import sleep
from Task3.resource import KeyPressModule as kp
from pyzbar.pyzbar import decode
import cv2
from threading import Thread
import numpy as np

kp.init()
tello = Tello()
tello.connect()
print("BATTERY:", tello.get_battery())

# define keyboard response
def getKeyboardInput():
    lr, fb, ud, yv = 0, 0, 0, 0
    speed = 50
    if kp.getKey("LEFT"):
        lr = -speed
    elif kp.getKey("RIGHT"):
        lr = speed
    if kp.getKey("UP"):
        fb = speed
    elif kp.getKey("DOWN"):
        fb = -speed
    if kp.getKey("w"):
        ud = speed
    elif kp.getKey("s"):
        ud = -speed
    if kp.getKey("a"):
        yv = -speed
    elif kp.getKey("d"):
        yv = speed
    if kp.getKey("q"):
        tello.land()
        # sleep(3)
    if kp.getKey("e"):
        tello.takeoff()
    if kp.getKey("i"):
        tello.flip_forward()
    if kp.getKey("k"):
        tello.flip_back()
    if kp.getKey("j"):
        tello.flip_left()
    if kp.getKey("l"):
        tello.flip_right()

    return [lr, fb, ud, yv]


# define video stream behavior
def stream():
    tello.streamon()
    # get video stream
    while True:
        level = tello.get_battery()
        img = tello.get_frame_read().frame
        img = cv2.resize(img, (360, 240))
        for barcode in decode(img):
            result = barcode.data.decode('utf-8')
            print(result)
            pts = np.array([barcode.polygon], np.int32)
            pts = pts.reshape((-1, 1, 2))
            # put boundary box on qr code
            cv2.polylines(img, [pts], True, (255, 0, 255), 5)
            pts2 = barcode.rect
            cv2.putText(img, result, (pts2[0], pts2[1]), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (255, 0, 255), 2)
        cv2.putText(img, str(level), (10, 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
        cv2.imshow("Tello", img)
        cv2.waitKey(1)



'''
Press E to take off, Q to landing
A and D control drone's yaw (clockwise or counter clockwise)
W and S control drone's up and down
Arrow UP and DOWN control drone's forward and backward
Arrow RIGHT and LEFT control drone's left and right
I, K, J, L doing flip
'''
if __name__ == "__main__":
    video_thread = Thread(target=stream)
    video_thread.start()
    while True:
        vals = getKeyboardInput()
        tello.send_rc_control(vals[0], vals[1], vals[2], vals[3])
        sleep(0.05)

## *4.Use different QR codes to control the drone*
#### STEP_0 : initialization tello

In [ ]:
from djitellopy import Tello
from pyzbar.pyzbar import decode
import cv2
import numpy as np

tello = Tello()
tello.connect()
print("BATTERY:", tello.get_battery())

#### STEP_1 : Define read QR doe behavior

In [ ]:
'''
Response tello command with input string
@:param cmd : command in string
'''

def getCodeInput(cmd):
    print("QR_CMD", cmd)
    while True:
        if cmd == "MOVE_LEFT":
            tello.move_left(30)
        elif cmd == "MOVE_RIGHT":
            tello.move_right(30)
        if cmd == "MOVE_FORWARD":
            tello.move_forward(30)
        elif cmd == "MOVE_BACK":
            tello.move_back(30)
        if cmd == "MOVE_UP":
            tello.move_up(30)
        elif cmd == "MOVE_DOWN":
            tello.move_down(30)
        if cmd == "ROTATE_COUNTER_CLOCKWISE":
            tello.rotate_counter_clockwise(90)
        elif cmd == "ROTATE_CLOCKWISE":
            tello.rotate_clockwise(90)
        if cmd == "LAND":
            tello.land()
        if cmd == "TAKE_OFF":
            tello.takeoff()

#### STEP_3 : Main thread

In [ ]:
if __name__ == "__main__":
    # enable tello stream
    tello.streamon()
    # command and count initialization
    cmd = "NULL"
    count = 0

    while True:
        level = tello.get_battery()
        img = tello.get_frame_read().frame
        img = cv2.resize(img, (360, 240))
        # resolve every qr code from image
        for barcode in decode(img):
            data = barcode.data.decode('utf-8')
            # save decode result in qr_cmd
            cmd = data
            count = count + 1
            pts = np.array([barcode.polygon], np.int32)
            pts = pts.reshape((-1, 1, 2))
            cv2.polylines(img, [pts], True, (255, 0, 255), 5)
            pts2 = barcode.rect
            cv2.putText(img, data, (pts2[0], pts2[1]), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (255, 0, 255), 2)
        cv2.putText(img, str(level), (10, 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)
        cv2.imshow("Tello", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            tello.land()
            break
        # response cmd
        if count > 30:
            getCodeInput(cmd)
            qr_count = 0
            qr_cmd = "NULL"

## *5.(optional) Control drone autonomously to stay at a certain relative position to the QR codes*

#### Define function in utlis module

In [ ]:
from djitellopy import Tello
import pyzbar.pyzbar as pyzbar
import cv2
import numpy as np

'''
Initialize tello drone
    1. create and connect Tello Drone
    2. enable video streaming
    3. set velocity to zero
    4. print current battery level
    5. return drone instance
'''


def initializeTello():
    tello = Tello()
    tello.connect()
    tello.for_back_velocity = 0
    tello.left_right_velocity = 0
    tello.up_down_velocity = 0
    tello.yaw_velocity = 0
    tello.speed = 0
    print("BATTERY:", tello.get_battery());
    tello.streamoff()
    tello.streamon()
    return tello


'''
Get Tello Drone Frame
    @param tello : drone instance
    @param w : width of frame
    @param h : height of frame
'''


def getTelloFrame(tello, w=360, h=240):
    frame = tello.get_frame_read().frame
    img = cv2.resize(frame, (w, h))
    level = tello.get_battery()
    cv2.putText(img, str(level), (10, 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0), 1)

    return img


'''
decode QR_code with pyzbar
    @param img : input image
    return img, qr_code position info
'''


def decode(img):
    pts2 = [0, 0, 0, 0]
    for barcode in pyzbar.decode(img):
        data = barcode.data.decode('utf-8')
        pts = np.array([barcode.polygon], np.int32)
        pts = pts.reshape((-1, 1, 2))
        cv2.polylines(img, [pts], True, (255, 0, 255), 3)
        pts2 = barcode.rect
        cv2.putText(img, data, (pts2[0], pts2[1]), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (255, 0, 255), 2)

    # if QR_Code exits
    if pts2[0] != 0 and pts2[1] != 0:
        return img, pts2
    else:
        return img, [0, 0, 0, 0]


'''
PID Control drone at relative position
    @param tello : drone instance
    @param pts : qr code position
    @param fw : width of frame
    @param fh : height of frame
    @param PID : PID control parameter (kp, kd, ki)
    @param pError : previous error
'''


def trackQRCode(tello, pts, fw, fh, pid, pError):
    # calculate error
    yaw_error = pts[0] - fw // 2  # stay at middle of width
    up_down_error = pts[1] - fh // 2 # stay at middle of height
    for_back_error = pts[2] * pts[3] - 83 * 79 # stay at 15 cm from QR Code (test result)

    # PID control Speed
    yaw_speed = pid[0] * yaw_error + pid[1] * (yaw_error-pError[0])
    up_down_speed = pid[0] * up_down_error + pid[1] * (up_down_error-pError[1])
    for_back_speed = pid[0] * for_back_error + pid[1] * (for_back_error-pError[2])

    # if QR code exits
    if pts[0] != 0 and pts[1] != 0:
        tello.yaw_velocity = yaw_speed
        tello.up_down_velocity = up_down_speed
        tello.for_back_velocity = for_back_speed
    else:
        tello.yaw_velocity = 0
        tello.left_right_velocity = 0
        tello.up_down_velocity = 0
        tello.yaw_velocity = 0
        yaw_error = 0
        up_down_error = 0
        for_back_error = 0

    if tello.send_rc_control:
        tello.send_rc_control(tello.left_right_velocity,
                              tello.for_back_velocity,
                              tello.up_down_velocity,
                              tello.yaw_velocity)

    return [yaw_error, up_down_error, for_back_error]


#### Main Program

In [ ]:
from utlis import *
import cv2

# frame size
w, h = 360, 250
# PID param kp ki kd
pid= [0.5, 0.5, 0]
# previous Error : yaw_error, up_down_error, for_back_error
pError = [0, 0, 0]


if __name__ == "__main__":
    # Tello Initialization
    tello = initializeTello()

    while True:
        img = getTelloFrame(tello, w, h)
        img, pts = decode(img)

        # print("W:", pts[2], "H", pts[3]) # W,H = 83, 79 at 15cm => 6557

        # control tello in PID way( only PI actually)
        pError = trackQRCode(tello, pts, w, h, pid, pError)

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            tello.land()
            break